In [1]:
#read in packages to be used
using CSV
using DataFrames
using FASTX
using StatsPlots

#by default Jupyter Notebook will limit the number of rows and columns when displaying a data frame. Override this behaviour by changing the values of the ENV["COLUMNS"]
ENV["COLUMNS"] = 400

400

In [2]:
#'reader' is a struct (an object) for reading .fasta files. Open() creates an IOStream which the FASTA.Reader will read from.
reader = FASTA.Reader(open("OR992133.1kb.fasta" , "r"))
#'record' is a struct for recording .fasta files. 
record = FASTA.Record()

#read and return
read!(reader, record)

#length of the sequence
genomelength = length(sequence(record)) - 1000
genome = sequence(record)[1:genomelength]
genomelength

LoadError: SystemError: opening file "C:/Users/20917397/Desktop/phylloglossum_editing/phylloglossum_drummondii_cp_1kb.fasta": No such file or directory

In [3]:
#the CSV package is creating a Table from the .gff file and the DataFrame package is creating a DataFrame from the Table.  
gff = DataFrame(CSV.File("OR992133.gff", delim="\t", comment ="#",
            header=["refID","chloe","type","start","end","score","strand","phase","annotation"]))
#print the first 6 rows of the DataFrame.
first(gff, 6)

LoadError: ArgumentError: "C:/Users/20917397/Desktop/phylloglossum_editing/phylloglossum_drummondii_cp_1kb.gff" is not a valid file or doesn't exist

In [4]:
#the CSV package is creating a Table from the .count file and the DataFrame package is creating a DataFrame from the Table. 
Ph1countscp = DataFrame(CSV.File("pdcp_r1.count", delim="\t", comment ="#"))

#drop the :total column from the DataFrame counts
Ph1countscp = combine(Ph1countscp, Not(:total))

#length of genome is equal to the number of row counts minus (last) 1000, the 1Kb overlap used in mapping. 
genomelength = nrow(Ph1countscp) - 1000

#sum extension counts. 
#the first 1000 rows of the DataFrame, from the 4th column to the last column, added
#to the rows from (genomelength+1) to the last row of the DataFrame (last 1000 rows), from column 4 to the last column. 
Ph1countscp[1:1000,4:end] .+= Ph1countscp[genomelength + 1:end,4:end]

#remove 1 Kb overlap (last 1000)
deleteat!(Ph1countscp, genomelength + 1:nrow(Ph1countscp))

#assigning IR1 and IR2
IR1 = gff[(occursin.("IR-1",gff.annotation)) .& (gff.type.=="repeat_region"),:]
IR2 = gff[(occursin.("IR-2",gff.annotation)) .& (gff.type.=="repeat_region"),:]

#combine counts in inverted repeats
Ph1countscp[IR1.start[1]:IR1.end[1],:fA] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:rA])
Ph1countscp[IR1.start[1]:IR1.end[1],:fC] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:rC])
Ph1countscp[IR1.start[1]:IR1.end[1],:fG] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:rG])
Ph1countscp[IR1.start[1]:IR1.end[1],:fU] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:rU])

Ph1countscp[IR1.start[1]:IR1.end[1],:rA] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:fA])
Ph1countscp[IR1.start[1]:IR1.end[1],:rC] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:fC])
Ph1countscp[IR1.start[1]:IR1.end[1],:rG] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:fG])
Ph1countscp[IR1.start[1]:IR1.end[1],:rU] .+= reverse(Ph1countscp[IR2.start[1]:IR2.end[1],:fU])

#combine counts in inverted repeats
#Ph1countscp[IR1.start[1]:IR1.end[1],4:end] .+= Ph1countscp[IR2.start[1]:IR2.end[1],4:end]

#Remove IR2
deleteat!(Ph1countscp, IR2.start[1]:IR2.end[1])

#forward count total
Ph1countscp.f_total = Ph1countscp.fA .+ Ph1countscp.fC .+ Ph1countscp.fG .+ Ph1countscp.fU

#reverse count total
Ph1countscp.r_total = Ph1countscp.rA .+ Ph1countscp.rC .+ Ph1countscp.rG .+ Ph1countscp.rU
#print the first 10 rows of the counts DataFrame
first(Ph1countscp, 10)

LoadError: ArgumentError: "C:/Users/20917397/Desktop/phylloglossum_editing/pdcp_r1.count" is not a valid file or doesn't exist

In [5]:
#Find tRNAs in gff file under "type"
tRNAs = gff[gff.type .== "tRNA",:]
#Set tRNA_intervals as the range between the start of the tRNA and the end of the tRNA.
tRNA_intervals = UnitRange.(tRNAs.start, tRNAs.end)
#filter and return everything excluding all tRNAs.
filter!(x -> !any(in.(x.pos, tRNA_intervals)), Ph1countscp)

#Find rRNAs in gff file under "type"
rRNAs = gff[gff.type .== "rRNA",:]
#Set rRNA_intervals as the range between the start of the rRNA and the end of the rRNA.
rRNA_intervals = UnitRange.(rRNAs.start, rRNAs.end)
#filter and return everything excluding all rRNAs.
filter!(x -> !any(in.(x.pos, rRNA_intervals)), Ph1countscp);

LoadError: UndefVarError: gff not defined

In [6]:
#the CSV package is creating a Table from the .count file and the DataFrame package is creating a DataFrame from the Table. 
Ph2countscp = DataFrame(CSV.File("pdcp_r2.count", delim="\t", comment ="#"))

#drop the :total column from the DataFrame counts
Ph2countscp = combine(Ph2countscp, Not(:total))

#sum extension counts. 
#the first 1000 rows of the DataFrame, from the 4th column to the last column, added
#to the rows from (genomelength+1) to the last row of the DataFrame (last 1000 rows), from column 4 to the last column. 
Ph2countscp[1:1000,4:end] .+= Ph2countscp[genomelength + 1:end,4:end]

#remove 1 Kb overlap (last 1000)
deleteat!(Ph2countscp, genomelength + 1:nrow(Ph2countscp))

#assigning IR1 and IR2
IR1 = gff[(occursin.("IR-1",gff.annotation)) .& (gff.type.=="repeat_region"),:]
IR2 = gff[(occursin.("IR-2",gff.annotation)) .& (gff.type.=="repeat_region"),:]

#combine counts in inverted repeats
Ph2countscp[IR1.start[1]:IR1.end[1],:fA] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:rA])
Ph2countscp[IR1.start[1]:IR1.end[1],:fC] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:rC])
Ph2countscp[IR1.start[1]:IR1.end[1],:fG] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:rG])
Ph2countscp[IR1.start[1]:IR1.end[1],:fU] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:rU])

Ph2countscp[IR1.start[1]:IR1.end[1],:rA] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:fA])
Ph2countscp[IR1.start[1]:IR1.end[1],:rC] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:fC])
Ph2countscp[IR1.start[1]:IR1.end[1],:rG] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:fG])
Ph2countscp[IR1.start[1]:IR1.end[1],:rU] .+= reverse(Ph2countscp[IR2.start[1]:IR2.end[1],:fU])

#Remove IR2
deleteat!(Ph2countscp, IR2.start[1]:IR2.end[1])

#forward count total
Ph2countscp.f_total = Ph2countscp.fA .+ Ph2countscp.fC .+ Ph2countscp.fG .+ Ph2countscp.fU

#reverse count total
Ph2countscp.r_total = Ph2countscp.rA .+ Ph2countscp.rC .+ Ph2countscp.rG .+ Ph2countscp.rU;

LoadError: ArgumentError: "C:/Users/20917397/Desktop/phylloglossum_editing/pdcp_r2.count" is not a valid file or doesn't exist

In [7]:
#Find tRNAs in gff file under "type"
tRNAs = gff[gff.type .== "tRNA",:]
#Set tRNA_intervals as the range between the start of the tRNA and the end of the tRNA.
tRNA_intervals = UnitRange.(tRNAs.start, tRNAs.end)
#filter and return everything excluding all tRNAs.
filter!(x -> !any(in.(x.pos, tRNA_intervals)), Ph2countscp)

#Find rRNAs in gff file under "type"
rRNAs = gff[gff.type .== "rRNA",:]
#Set rRNA_intervals as the range between the start of the rRNA and the end of the rRNA.
rRNA_intervals = UnitRange.(rRNAs.start, rRNAs.end)
#filter and return everything excluding all rRNAs.
filter!(x -> !any(in.(x.pos, rRNA_intervals)), Ph2countscp);

LoadError: UndefVarError: gff not defined

In [8]:
#the CSV package is creating a Table from the .count file and the DataFrame package is creating a DataFrame from the Table. 
Ph3countscp = DataFrame(CSV.File("pdcp_r3.count", delim="\t", comment ="#"))

#drop the :total column from the DataFrame counts
Ph3countscp = combine(Ph3countscp, Not(:total))

#sum extension counts. 
#the first 1000 rows of the DataFrame, from the 4th column to the last column, added
#to the rows from (genomelength+1) to the last row of the DataFrame (last 1000 rows), from column 4 to the last column. 
Ph3countscp[1:1000,4:end] .+= Ph3countscp[genomelength + 1:end,4:end]

#remove 1 Kb overlap (last 1000)
deleteat!(Ph3countscp, genomelength + 1:nrow(Ph3countscp))

#assigning IR1 and IR2
IR1 = gff[(occursin.("IR-1",gff.annotation)) .& (gff.type.=="repeat_region"),:]
IR2 = gff[(occursin.("IR-2",gff.annotation)) .& (gff.type.=="repeat_region"),:]

#combine counts in inverted repeats
Ph3countscp[IR1.start[1]:IR1.end[1],:fA] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:rA])
Ph3countscp[IR1.start[1]:IR1.end[1],:fC] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:rC])
Ph3countscp[IR1.start[1]:IR1.end[1],:fG] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:rG])
Ph3countscp[IR1.start[1]:IR1.end[1],:fU] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:rU])

Ph3countscp[IR1.start[1]:IR1.end[1],:rA] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:fA])
Ph3countscp[IR1.start[1]:IR1.end[1],:rC] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:fC])
Ph3countscp[IR1.start[1]:IR1.end[1],:rG] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:fG])
Ph3countscp[IR1.start[1]:IR1.end[1],:rU] .+= reverse(Ph3countscp[IR2.start[1]:IR2.end[1],:fU])

#Remove IR2
deleteat!(Ph3countscp, IR2.start[1]:IR2.end[1])

#forward count total
Ph3countscp.f_total = Ph3countscp.fA .+ Ph3countscp.fC .+ Ph3countscp.fG .+ Ph3countscp.fU

#reverse count total
Ph3countscp.r_total = Ph3countscp.rA .+ Ph3countscp.rC .+ Ph3countscp.rG .+ Ph3countscp.rU;

LoadError: ArgumentError: "C:/Users/20917397/Desktop/phylloglossum_editing/pdcp_r3.count" is not a valid file or doesn't exist

In [9]:
#Find tRNAs in gff file under "type"
tRNAs = gff[gff.type .== "tRNA",:]
#Set tRNA_intervals as the range between the start of the tRNA and the end of the tRNA.
tRNA_intervals = UnitRange.(tRNAs.start, tRNAs.end)
#filter and return everything excluding all tRNAs.
filter!(x -> !any(in.(x.pos, tRNA_intervals)), Ph3countscp)

#Find rRNAs in gff file under "type"
rRNAs = gff[gff.type .== "rRNA",:]
#Set rRNA_intervals as the range between the start of the rRNA and the end of the rRNA.
rRNA_intervals = UnitRange.(rRNAs.start, rRNAs.end)
#filter and return everything excluding all rRNAs.
filter!(x -> !any(in.(x.pos, rRNA_intervals)), Ph3countscp);

LoadError: UndefVarError: gff not defined

In [10]:
#the CSV package is creating a Table from the .count file and the DataFrame package is creating a DataFrame from the Table. 
PhFcountscp = DataFrame(CSV.File("pdcp_f.count", delim="\t", comment ="#"))

#drop the :total column from the DataFrame counts
PhFcountscp = combine(PhFcountscp, Not(:total))

#sum extension counts. 
#the first 1000 rows of the DataFrame, from the 4th column to the last column, added
#to the rows from (genomelength+1) to the last row of the DataFrame (last 1000 rows), from column 4 to the last column. 
PhFcountscp[1:1000,4:end] .+= PhFcountscp[genomelength + 1:end,4:end]

#remove 1 Kb overlap (last 1000)
deleteat!(PhFcountscp, genomelength + 1:nrow(PhFcountscp))

#assigning IR1 and IR2
IR1 = gff[(occursin.("IR-1",gff.annotation)) .& (gff.type.=="repeat_region"),:]
IR2 = gff[(occursin.("IR-2",gff.annotation)) .& (gff.type.=="repeat_region"),:]

#combine counts in inverted repeats
PhFcountscp[IR1.start[1]:IR1.end[1],:fA] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:rA])
PhFcountscp[IR1.start[1]:IR1.end[1],:fC] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:rC])
PhFcountscp[IR1.start[1]:IR1.end[1],:fG] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:rG])
PhFcountscp[IR1.start[1]:IR1.end[1],:fU] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:rU])

PhFcountscp[IR1.start[1]:IR1.end[1],:rA] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:fA])
PhFcountscp[IR1.start[1]:IR1.end[1],:rC] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:fC])
PhFcountscp[IR1.start[1]:IR1.end[1],:rG] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:fG])
PhFcountscp[IR1.start[1]:IR1.end[1],:rU] .+= reverse(PhFcountscp[IR2.start[1]:IR2.end[1],:fU])

#Remove IR2
deleteat!(PhFcountscp, IR2.start[1]:IR2.end[1])

#forward count total
PhFcountscp.f_total = PhFcountscp.fA .+ PhFcountscp.fC .+ PhFcountscp.fG .+ PhFcountscp.fU

#reverse count total
PhFcountscp.r_total = PhFcountscp.rA .+ PhFcountscp.rC .+ PhFcountscp.rG .+ PhFcountscp.rU;

LoadError: ArgumentError: "C:/Users/20917397/Desktop/phylloglossum_editing/pdcp_f.count" is not a valid file or doesn't exist

In [11]:
#Find tRNAs in gff file under "type"
tRNAs = gff[gff.type .== "tRNA",:]
#Set tRNA_intervals as the range between the start of the tRNA and the end of the tRNA.
tRNA_intervals = UnitRange.(tRNAs.start, tRNAs.end)
#filter and return everything excluding all tRNAs.
filter!(x -> !any(in.(x.pos, tRNA_intervals)), PhFcountscp)

#Find rRNAs in gff file under "type"
rRNAs = gff[gff.type .== "rRNA",:]
#Set rRNA_intervals as the range between the start of the rRNA and the end of the rRNA.
rRNA_intervals = UnitRange.(rRNAs.start, rRNAs.end)
#filter and return everything excluding all rRNAs.
filter!(x -> !any(in.(x.pos, rRNA_intervals)), PhFcountscp);

LoadError: UndefVarError: gff not defined

In [12]:
#DataFrames counts plot (position, forward strand, colour, label, size (in pixels)). 
@df Ph1countscp plot(:pos, :f_total, color=:red, label="Ph1", size=(1000,400))
@df Ph2countscp plot!(:pos, :f_total, color=:green, label="Ph2")
@df Ph3countscp plot!(:pos, :f_total, color=:blue, label="Ph3")
@df PhFcountscp plot!(:pos, :f_total, color=:orchid, label="PhF")
#DataFrames counts plot (position, reverse strand, colour, no label, size (in pixels)).
@df Ph1countscp plot!(:pos, -:r_total, color=:red, label=false)
@df Ph2countscp plot!(:pos, -:r_total, color=:green, label=false)
@df Ph3countscp plot!(:pos, -:r_total, color=:blue, label=false)
@df PhFcountscp plot!(:pos, -:r_total, color=:orchid, label=false);

LoadError: UndefVarError: Ph1countscp not defined

In [13]:
#the CSV package is creating a Table from the .gff file and the DataFrame package is creating a DataFrame from the Table. 
dcountscp = DataFrame(CSV.File("pdcp_d.count", delim="\t", comment ="#"))

#drop the :total column from the DataFrame counts
dcountscp = combine(dcountscp, Not(:total))

#length of genome is equal to the number of row counts minus (last) 1000, the 1Kb overlap used in mapping. 
genomelength = nrow(dcountscp) - 1000

#sum extension counts. 
#the first 1000 rows of the DataFrame, from the 4th column to the last column, added
#to the rows from (genomelength+1) to the last row of the DataFrame (last 1000 rows), from column 4 to the last column. 
dcountscp[1:1000,4:end] .+= dcountscp[genomelength + 1:end,4:end]

#remove 1 Kb overlap (last 1000)
deleteat!(dcountscp, genomelength + 1:nrow(dcountscp))

#combine counts in inverted repeats
dcountscp[IR1.start[1]:IR1.end[1],:fA] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:rA])
dcountscp[IR1.start[1]:IR1.end[1],:fC] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:rC])
dcountscp[IR1.start[1]:IR1.end[1],:fG] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:rG])
dcountscp[IR1.start[1]:IR1.end[1],:fU] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:rU])

dcountscp[IR1.start[1]:IR1.end[1],:rA] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:fA])
dcountscp[IR1.start[1]:IR1.end[1],:rC] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:fC])
dcountscp[IR1.start[1]:IR1.end[1],:rG] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:fG])
dcountscp[IR1.start[1]:IR1.end[1],:rU] .+= reverse(dcountscp[IR2.start[1]:IR2.end[1],:fU])

#Remove IR2
deleteat!(dcountscp, IR2.start[1]:IR2.end[1])

#forward count total
dcountscp.f_total = dcountscp.fA .+ dcountscp.fC .+ dcountscp.fG .+ dcountscp.fU

#reverse count total
dcountscp.r_total = dcountscp.rA .+ dcountscp.rC .+ dcountscp.rG .+ dcountscp.rU;

LoadError: ArgumentError: "C:/Users/20917397/Desktop/phylloglossum_editing/pdcp_d.count" is not a valid file or doesn't exist

In [14]:
#Find tRNAs in gff file under "type"
tRNAs = gff[gff.type .== "tRNA",:]
#Set tRNA_intervals as the range between the start of the tRNA and the end of the tRNA.
tRNA_intervals = UnitRange.(tRNAs.start, tRNAs.end)
#filter and return everything excluding all tRNAs.
filter!(x -> !any(in.(x.pos, tRNA_intervals)), dcountscp)

#Find rRNAs in gff file under "type"
rRNAs = gff[gff.type .== "rRNA",:]
#Set rRNA_intervals as the range between the start of the rRNA and the end of the rRNA.
rRNA_intervals = UnitRange.(rRNAs.start, rRNAs.end)
#filter and return everything excluding all rRNAs.
filter!(x -> !any(in.(x.pos, rRNA_intervals)), dcountscp);

LoadError: UndefVarError: gff not defined

In [15]:
#DataFrames counts plot (position, forward strand), label, size (in pixels) 
@df dcountscp plot(:pos, :f_total, label="DNA", size=(1000,400))
#DataFrames counts plot (position, inverse: reverse strand), no label, size (in pixels) 
@df dcountscp plot!(:pos, -:r_total, label=false, size=(1000,400));

LoadError: UndefVarError: dcountscp not defined

In [16]:
#number of C. filtered when the reference is either C or G.
Ph1Ccountscp = filter(x -> x.ref ∈ ["C", "G"], Ph1countscp)
#create extra columns with zeros for edited and unedited counts shown on both the forward and reverse strands.
Ph1Ccountscp.edited = zeros(Int, nrow(Ph1Ccountscp))
Ph1Ccountscp.unedited = zeros(Int, nrow(Ph1Ccountscp))
Ph1Ccountscp.rsedited = zeros(Int, nrow(Ph1Ccountscp))
Ph1Ccountscp.rsunedited = zeros(Int, nrow(Ph1Ccountscp))

#forward strand C-U editing
#total C. edited on forward strand (C on reference) is equal to the total U on the forward strand (when ref = C).
Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :edited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :fU]
#total C. edited on forward strand (C on reference) is equal to the total A on the reverse strand (when ref = C).
Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :rsedited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :rA]
#total C. unedited on the forward strand (C on reference) is equal to the total on the forward strand, minus the number of edited shown on the forward strand (when ref = C).
Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :unedited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :f_total] .- Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :edited]
#total C. unedited on the forward strand (C on reference) is equal to the total on the reverse strand, minus the number of edited shown on the reverse strand (when ref = C).
Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :rsunedited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :r_total] .- Ph1Ccountscp[Ph1Ccountscp.ref .== "C", :rsedited]

#reverse strand C-U editing
#total C. edited on the reverse strand (G on reference) is equal to the total U on the reverse strand (when ref = G). 
Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :edited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :rU]
#total C. edited on the reverse strand (G on reference) is equal to the total A on the forward strand (when ref = G). 
Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :rsedited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :fA]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the reverse strand, minus the number of edited shown on the forward strand (when ref = G).
Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :unedited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :r_total] .- Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :edited]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the forward strand, minus the number of edited shown on the reverse strand (when ref = G). 
Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :rsunedited] = Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :f_total] .- Ph1Ccountscp[Ph1Ccountscp.ref .== "G", :rsedited]

#total counts = the sum of edited and unedited shown on the forward strand.
Ph1Ccountscp.total = Ph1Ccountscp.edited .+ Ph1Ccountscp.unedited
#total counts = the sum of edited and unedited shown on the reverse strand.
Ph1Ccountscp.rstotal = Ph1Ccountscp.rsedited .+ Ph1Ccountscp.rsunedited
#what proportion of the total is edited (shown on the forward strand).
Ph1Ccountscp.editp = Ph1Ccountscp.edited ./ Ph1Ccountscp.total
#what proportion of the total is edited (shown on the reverse strand).
Ph1Ccountscp.rseditp = Ph1Ccountscp.rsedited ./ Ph1Ccountscp.rstotal
#return filter for reads larger or equal to 100.
filter!(x -> x.total >= 100, Ph1Ccountscp);
first(Ph1countscp, 10)

LoadError: UndefVarError: Ph1countscp not defined

In [17]:
#number of C. filtered when the reference is either C or G.
Ph2Ccountscp = filter(x -> x.ref ∈ ["C", "G"], Ph2countscp)
#create extra columns with zeros for edited and unedited counts shown on both the forward and reverse strands.
Ph2Ccountscp.edited = zeros(Int, nrow(Ph2Ccountscp))
Ph2Ccountscp.unedited = zeros(Int, nrow(Ph2Ccountscp))
Ph2Ccountscp.rsedited = zeros(Int, nrow(Ph2Ccountscp))
Ph2Ccountscp.rsunedited = zeros(Int, nrow(Ph2Ccountscp))

#forward strand C-U editing
#total C. edited on forward strand (C on reference) is equal to the total U on the forward strand (when ref = C).
Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :edited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :fU]
#total C. edited on forward strand (C on reference) is equal to the total A on the reverse strand (when ref = C).
Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :rsedited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :rA]
#total C. unedited on the forward strand (C on reference) is equal to the total on the forward strand, minus the number of edited shown on the forward strand (when ref = C).
Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :unedited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :f_total] .- Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :edited]
#total C. unedited on the forward strand (C on reference) is equal to the total on the reverse strand, minus the number of edited shown on the reverse strand (when ref = C).
Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :rsunedited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :r_total] .- Ph2Ccountscp[Ph2Ccountscp.ref .== "C", :rsedited]

#reverse strand C-U editing
#total C. edited on reverse strand (G on reference) is equal to the total U on the reverse strand (when ref = G). 
Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :edited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :rU]
#total C. edited on the reverse strand (G on reference) is equal to the total A on the forward strand (when ref = G). 
Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :rsedited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :fA]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the reverse strand, minus the number of edited shown on the forward strand (when ref = G).
Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :unedited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :r_total] .- Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :edited]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the forward strand, minus the number of edited shown on the reverse strand (when ref = G).
Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :rsunedited] = Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :f_total] .- Ph2Ccountscp[Ph2Ccountscp.ref .== "G", :rsedited]

#total counts = the sum of edited and unedited shown on the forward strand.
Ph2Ccountscp.total = Ph2Ccountscp.edited .+ Ph2Ccountscp.unedited
#total counts = the sum of edited and unedited shown on the reverse strand.
Ph2Ccountscp.rstotal = Ph2Ccountscp.rsedited .+ Ph2Ccountscp.rsunedited
#what proportion of the total is edited (shown on the forward strand).
Ph2Ccountscp.editp = Ph2Ccountscp.edited ./ Ph2Ccountscp.total
#what proportion of the total is edited (shown on the reverse strand).
Ph2Ccountscp.rseditp = Ph2Ccountscp.rsedited ./ Ph2Ccountscp.rstotal
#return filter for reads larger or equal to 100.
filter!(x -> x.total >= 100, Ph2Ccountscp);

LoadError: UndefVarError: Ph2countscp not defined

In [18]:
#number of C. filtered when the reference is either C or G.
Ph3Ccountscp = filter(x -> x.ref ∈ ["C", "G"], Ph3countscp)
#create extra columns with zeros for edited and unedited counts shown on both the forward and reverse strands.
Ph3Ccountscp.edited = zeros(Int, nrow(Ph3Ccountscp))
Ph3Ccountscp.unedited = zeros(Int, nrow(Ph3Ccountscp))
Ph3Ccountscp.rsedited = zeros(Int, nrow(Ph3Ccountscp))
Ph3Ccountscp.rsunedited = zeros(Int, nrow(Ph3Ccountscp))

#forward strand C-U editing
#total C. edited on forward strand (C on reference) is equal to the total U on the forward strand (when ref = C).
Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :edited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :fU]
#total C. edited on forward strand (C on reference) is equal to the total A on the reverse strand (when ref = C).
Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :rsedited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :rA]
#total C. unedited on the forward strand (C on reference) is equal to the total on the forward strand, minus the number of edited shown on the forward strand (when ref = C).
Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :unedited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :f_total] .- Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :edited]
#total C. unedited on the forward strand (C on reference) is equal to the total on the reverse strand, minus the number of edited shown on the reverse strand (when ref = C).
Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :rsunedited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :r_total] .- Ph3Ccountscp[Ph3Ccountscp.ref .== "C", :rsedited]

#reverse strand C-U editing
#total C. edited on reverse strand (G on reference) is equal to the total U on the reverse strand (when ref = G). 
Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :edited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :rU]
#total C. edited on the reverse strand (G on reference) is equal to the total A on the forward strand (when ref = G). 
Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :rsedited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :fA]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the reverse strand, minus the number of edited shown on the forward strand (when ref = G).
Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :unedited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :r_total] .- Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :edited]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the forward strand, minus the number of edited shown on the reverse strand (when ref = G).
Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :rsunedited] = Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :f_total] .- Ph3Ccountscp[Ph3Ccountscp.ref .== "G", :rsedited]

#total counts = the sum of edited and unedited shown on the forward strand.
Ph3Ccountscp.total = Ph3Ccountscp.edited .+ Ph3Ccountscp.unedited
#total counts = the sum of edited and unedited shown on the reverse strand.
Ph3Ccountscp.rstotal = Ph3Ccountscp.rsedited .+ Ph3Ccountscp.rsunedited
#what proportion of the total is edited (shown on the forward strand).
Ph3Ccountscp.editp = Ph3Ccountscp.edited ./ Ph3Ccountscp.total
#what proportion of the total is edited (shown on the reverse strand).
Ph3Ccountscp.rseditp = Ph3Ccountscp.rsedited ./ Ph3Ccountscp.rstotal
#return filter for reads larger or equal to 100.
filter!(x -> x.total >= 100, Ph3Ccountscp);

LoadError: UndefVarError: Ph3countscp not defined

In [19]:
#number of C. filtered when the reference is either C or G.
PhFCcountscp = filter(x -> x.ref ∈ ["C", "G"], PhFcountscp)
#create extra columns with zeros for edited and unedited counts shown on both the forward and reverse strands.
PhFCcountscp.edited = zeros(Int, nrow(PhFCcountscp))
PhFCcountscp.unedited = zeros(Int, nrow(PhFCcountscp))
PhFCcountscp.rsedited = zeros(Int, nrow(PhFCcountscp))
PhFCcountscp.rsunedited = zeros(Int, nrow(PhFCcountscp))

#forward strand C-U editing
#total C. edited on forward strand (C on reference) is equal to the total U on the forward strand (when ref = C).
PhFCcountscp[PhFCcountscp.ref .== "C", :edited] = PhFCcountscp[PhFCcountscp.ref .== "C", :fU]
#total C. edited on forward strand (C on reference) is equal to the total A on the reverse strand (when ref = C).
PhFCcountscp[PhFCcountscp.ref .== "C", :rsedited] = PhFCcountscp[PhFCcountscp.ref .== "C", :rA]
#total C. unedited on the forward strand (C on reference) is equal to the total on the forward strand, minus the number of edited shown on the forward strand (when ref = C).
PhFCcountscp[PhFCcountscp.ref .== "C", :unedited] = PhFCcountscp[PhFCcountscp.ref .== "C", :f_total] .- PhFCcountscp[PhFCcountscp.ref .== "C", :edited]
#total C. unedited on the forward strand (C on reference) is equal to the total on the reverse strand, minus the number of edited shown on the reverse strand (when ref = C).
PhFCcountscp[PhFCcountscp.ref .== "C", :rsunedited] = PhFCcountscp[PhFCcountscp.ref .== "C", :r_total] .- PhFCcountscp[PhFCcountscp.ref .== "C", :rsedited]

#reverse strand C-U editing
#total C. edited on reverse strand (G on reference) is equal to the total U on the reverse strand (when ref = G). 
PhFCcountscp[PhFCcountscp.ref .== "G", :edited] = PhFCcountscp[PhFCcountscp.ref .== "G", :rU]
#total C. edited on the reverse strand (G on reference) is equal to the total A on the forward strand (when ref = G). 
PhFCcountscp[PhFCcountscp.ref .== "G", :rsedited] = PhFCcountscp[PhFCcountscp.ref .== "G", :fA]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the reverse strand, minus the number of edited shown on the forward strand (when ref = G).
PhFCcountscp[PhFCcountscp.ref .== "G", :unedited] = PhFCcountscp[PhFCcountscp.ref .== "G", :r_total] .- PhFCcountscp[PhFCcountscp.ref .== "G", :edited]
#total C. unedited on the reverse strand (G on reference) is equal to the total on the forward strand, minus the number of edited shown on the reverse strand (when ref = G).
PhFCcountscp[PhFCcountscp.ref .== "G", :rsunedited] = PhFCcountscp[PhFCcountscp.ref .== "G", :f_total] .- PhFCcountscp[PhFCcountscp.ref .== "G", :rsedited]

#total counts = the sum of edited and unedited shown on the forward strand.
PhFCcountscp.total = PhFCcountscp.edited .+ PhFCcountscp.unedited
#total counts = the sum of edited and unedited shown on the reverse strand.
PhFCcountscp.rstotal = PhFCcountscp.rsedited .+ PhFCcountscp.rsunedited
#what proportion of the total is edited (shown on the forward strand).
PhFCcountscp.editp = PhFCcountscp.edited ./ PhFCcountscp.total
#what proportion of the total is edited (shown on the reverse strand).
PhFCcountscp.rseditp = PhFCcountscp.rsedited ./ PhFCcountscp.rstotal
#return filter for reads larger or equal to 100.
filter!(x -> x.total >= 100, PhFCcountscp);

LoadError: UndefVarError: PhFcountscp not defined

In [20]:
#number of C. filtered when the reference is either C or G.
dCcountscp = filter(x -> x.ref ∈ ["C", "G"], dcountscp)
#create extra columns with zeros for both edited and unedited counts.
dCcountscp.edited = zeros(Int, nrow(dCcountscp))
dCcountscp.unedited = zeros(Int, nrow(dCcountscp))

#forward strand C-U editing
#total C. edited on forward strand (C on reference) is equal to total U on the forward strand (when ref = C).
dCcountscp[dCcountscp.ref .== "C", :edited] = dCcountscp[dCcountscp.ref .== "C", :fU]
#total C. unedited on the forward strand is equal to the total on the forward strand (when ref = C) minus the number of edited (when ref = C)
dCcountscp[dCcountscp.ref .== "C", :unedited] = dCcountscp[dCcountscp.ref .== "C", :f_total] .- dCcountscp[dCcountscp.ref .== "C", :edited]

#reverse strand C-U editing
#total C. edited on reverse strand (G on reference) is equal to total U on the reverse strand (when ref = G). 
dCcountscp[dCcountscp.ref .== "G", :edited] = dCcountscp[dCcountscp.ref .== "G", :rU]
#total C. unedited on the reverse strand is equal to the total on the reverse strand (when ref = G) minus the number of edited (when ref = G)
dCcountscp[dCcountscp.ref .== "G", :unedited] = dCcountscp[dCcountscp.ref .== "G", :r_total] .- dCcountscp[dCcountscp.ref .== "G", :edited]

#total counts = the sum of edited and unedited.
dCcountscp.total = dCcountscp.edited .+ dCcountscp.unedited
#what proportion of the total is edited.
dCcountscp.editp = dCcountscp.edited ./ dCcountscp.total;

LoadError: UndefVarError: dcountscp not defined

In [21]:
#read in packages to be used
using HypothesisTests
using MultipleTesting

In [22]:
#use innerjoin function to join the first RNA counts to the DNA counts based on position.
#makeunique : if false (the default), an error will be raised if duplicate names are found; if true, duplicate names will be suffixed with _i (i starting at 1 for the first duplicate).
Ph1cp = innerjoin(Ph1Ccountscp, dCcountscp, on = :pos, makeunique=true)
#perform Fisher's exact test of the null hypothesis that the success probabilities ``a/c`` and ``b/d`` are equal.
#add 1 to all results so that no zeros are produced, zeros will produce an error message.
fisher = FisherExactTest.(Ph1cp.edited .+ 1, Ph1cp.unedited .+ 1, Ph1cp.edited_1 .+ 1, Ph1cp.unedited_1 .+ 1);
#compute the p-value for a given Fisher exact test (right-tailed).
#the one-sided p-values are based on Fisher's non-central hypergeometric distribution.
Ph1cp.fisherp = pvalue.(fisher; tail = :right)
#use Benjamini-Hochberg to adjust the individual p-value for each nucleotide to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
Ph1cp.fBH = adjust(Ph1cp.fisherp, BenjaminiHochberg());
#fisherω is equal to the log of the 'conditional maximum likehood estimate of the odds ratio'.
Ph1cp.fisherω = [log(x.ω) for x in fisher]
#sort in reverse order (rev=true), printing the first 6 rows of the DataFrame.
first(sort(Ph1cp, :fisherω, rev=true), 6)

LoadError: UndefVarError: Ph1Ccountscp not defined

In [23]:
#use innerjoin function to join the first RNA counts to the DNA counts based on position.
#makeunique : if false (the default), an error will be raised if duplicate names are found; if true, duplicate names will be suffixed with _i (i starting at 1 for the first duplicate).
Ph2cp = innerjoin(Ph2Ccountscp, dCcountscp, on = :pos, makeunique=true)
#perform Fisher's exact test of the null hypothesis that the success probabilities ``a/c`` and ``b/d`` are equal.
#add 1 to all results so that no zeros are produced, zeros will produce an error message.
fisher = FisherExactTest.(Ph2cp.edited .+ 1, Ph2cp.unedited .+ 1, Ph2cp.edited_1 .+ 1, Ph2cp.unedited_1 .+ 1);
#compute the p-value for a given Fisher exact test (right-tailed).
#the one-sided p-values are based on Fisher's non-central hypergeometric distribution.
Ph2cp.fisherp = pvalue.(fisher; tail = :right)
#use Benjamini-Hochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
Ph2cp.fBH = adjust(Ph2cp.fisherp, BenjaminiHochberg());
#fisherω is equal to the log of the 'conditional maximum likehood estimate of the odds ratio'.
Ph2cp.fisherω = [log(x.ω) for x in fisher];

LoadError: UndefVarError: Ph2Ccountscp not defined

In [24]:
#use innerjoin function to join the first RNA counts to the DNA counts based on position.
#makeunique : if false (the default), an error will be raised if duplicate names are found; if true, duplicate names will be suffixed with _i (i starting at 1 for the first duplicate).
Ph3cp = innerjoin(Ph3Ccountscp, dCcountscp, on = :pos, makeunique=true)
#perform Fisher's exact test of the null hypothesis that the success probabilities ``a/c`` and ``b/d`` are equal.
#add 1 to all results so that no zeros are produced, zeros will produce an error message.
fisher = FisherExactTest.(Ph3cp.edited .+ 1, Ph3cp.unedited .+ 1, Ph3cp.edited_1 .+ 1, Ph3cp.unedited_1 .+ 1);
#compute the p-value for a given Fisher exact test (right-tailed).
#the one-sided p-values are based on Fisher's non-central hypergeometric distribution.
Ph3cp.fisherp = pvalue.(fisher; tail = :right)
#use Benjamini-Hochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
Ph3cp.fBH = adjust(Ph3cp.fisherp, BenjaminiHochberg());
#fisherω is equal to the log of the 'conditional maximum likehood estimate of the odds ratio'.
Ph3cp.fisherω = [log(x.ω) for x in fisher];

LoadError: UndefVarError: Ph3Ccountscp not defined

In [25]:
#use innerjoin function to join the first RNA counts to the DNA counts based on position.
#makeunique : if false (the default), an error will be raised if duplicate names are found; if true, duplicate names will be suffixed with _i (i starting at 1 for the first duplicate).
PhFcp = innerjoin(PhFCcountscp, dCcountscp, on = :pos, makeunique=true)
#perform Fisher's exact test of the null hypothesis that the success probabilities ``a/c`` and ``b/d`` are equal.
#add 1 to all results so that no zeros are produced, zeros will produce an error message.
fisher = FisherExactTest.(PhFcp.edited .+ 1, PhFcp.unedited .+ 1, PhFcp.edited_1 .+ 1, PhFcp.unedited_1 .+ 1);
#compute the p-value for a given Fisher exact test (right-tailed).
#the one-sided p-values are based on Fisher's non-central hypergeometric distribution.
PhFcp.fisherp = pvalue.(fisher; tail = :right)
#use Benjamini-Hochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
PhFcp.fBH = adjust(PhFcp.fisherp, BenjaminiHochberg());
#fisherω is equal to the log of the 'conditional maximum likehood estimate of the odds ratio'.
PhFcp.fisherω = [log(x.ω) for x in fisher];

LoadError: UndefVarError: PhFCcountscp not defined

In [26]:
#print line as editing sites in Ph1: the sum of all editing sites after adjusted (Benjamini-Hochberg), p-value less than 0.05.
println("editing sites in Ph1cp: ", sum(Ph1cp.fBH .< 0.05))
#print line as editing sites in Ph2: the sum of all editing sites after adjusted (Benjamini-Hochberg), p-value less than 0.05.
println("editing sites in Ph2cp: ", sum(Ph2cp.fBH .< 0.05))
#print line as editing sites in Ph3: the sum of all editing sites after adjusted (Benjamini-Hochberg), p-value less than 0.05.
println("editing sites in Ph3cp: ", sum(Ph3cp.fBH .< 0.05))
#print line as editing sites in PhF: the sum of all editing sites after adjusted (Benjamini-Hochberg), p-value less than 0.05.
println("editing sites in PhFcp: ", sum(PhFcp.fBH .< 0.05))

LoadError: UndefVarError: Ph1cp not defined

In [27]:
#use innerjoin function to 'join' Ph1 and Ph2 at the position column.
#makeunique=true, duplicate names will be suffixed with _i (i=1 for the first duplicate).
allPhcp = innerjoin(Ph1cp, Ph2cp, on = :pos, makeunique=true)
#use innerjoin function to 'join' allPh(Ph1, Ph2) and Ph3 at the position column.
#makeunique=true, duplicate names will be suffixed with _i (i=2 for the second duplicate).
allPhcp = innerjoin(allPhcp, Ph3cp, on = :pos, makeunique=true)
#use innerjoin function to 'join' allPh(Ph1, Ph2, Ph3) and Ph4 at the position column.
#makeunique=true, duplicate names will be suffixed with _i (i=3 for the third duplicate).
allPhcp = innerjoin(allPhcp, PhFcp, on = :pos, makeunique=true)
#sum allPh.fBH_1, allPh.fBH_2, allPh.fBH_3 (with a p-value less than 0.05).
sum((allPhcp.fBH .< 0.05) .& (allPhcp.fBH_1 .< 0.05) .& (allPhcp.fBH_2 .< 0.05) .& (allPhcp.fBH_3 .< 0.05))

LoadError: UndefVarError: Ph1cp not defined

In [28]:
#make a scatter plot from the DataFrame containing allPh.fBH_1, allPh.fBH_2, allPh.fBH_3 (with a p-value less than 0.05). 
#The scatter plot will consist of position and proportion edited. 
#The ms (marker size), msw (marker stroke width) and alpha (softens colour) are set, there is no legend
# the x and y axes are defined as: x axis="genome position", y axis="proportion of edited transcripts"
editing_sites_fisher_plot = @df allPhcp[(allPhcp.fBH .< 0.05) .& (allPhcp.fBH_1 .< 0.05) .& (allPhcp.fBH_2 .< 0.05) .& (allPhcp.fBH_3 .< 0.05),:] scatter(:pos, :editp, ms=4, msw=0,
    alpha=2, legend=false, xguide="Chloroplast Genome Position", yguide="Proportion of Edited Transcripts")

LoadError: UndefVarError: allPhcp not defined

In [29]:
savefig("editing_sites_C-U_cp_fisher_sites.svg")

LoadError: No current plot/subplot

In [30]:
#editing sites are defined as all sites in Ph1, Ph2, Ph3 and PhF: All editing sites after adjusted (Benjamini-Hochberg), p-value less than 0.05.
editing_sites_fisher = allPhcp[(allPhcp.fBH .< 0.05) .& (allPhcp.fBH_1 .< 0.05) .& (allPhcp.fBH_2 .< 0.05) .& (allPhcp.fBH_3 .< 0.05),:]

LoadError: UndefVarError: allPhcp not defined

In [31]:
CSV.write("editing_sites_C-U_cp_fisher.csv", editing_sites_fisher)

LoadError: UndefVarError: editing_sites_fisher not defined

In [32]:
#run a binomial test with a pvalue of 0.05 for Ph1 counts, single-tailed to the right.
Ph1Ccountscp.binomial = pvalue.(BinomialTest.(Ph1Ccountscp.edited, Ph1Ccountscp.total, 0.05); tail = :right)
#use BenjaminiHochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
Ph1Ccountscp.bBH = adjust(Ph1Ccountscp.binomial, BenjaminiHochberg())

#run a binomial test with a pvalue of 0.05 for Ph2 counts, single-tailed to the right.
Ph2Ccountscp.binomial = pvalue.(BinomialTest.(Ph2Ccountscp.edited, Ph2Ccountscp.total, 0.05); tail = :right)
#use BenjaminiHochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
Ph2Ccountscp.bBH = adjust(Ph2Ccountscp.binomial, BenjaminiHochberg())

#run a binomial test with a pvalue of 0.05 for Ph3 counts, single-tailed to the right.
Ph3Ccountscp.binomial = pvalue.(BinomialTest.(Ph3Ccountscp.edited, Ph3Ccountscp.total, 0.05); tail = :right)
#use BenjaminiHochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
Ph3Ccountscp.bBH = adjust(Ph3Ccountscp.binomial, BenjaminiHochberg());

#run a binomial test with a pvalue of 0.05 for PhF counts, single-tailed to the right.
PhFCcountscp.binomial = pvalue.(BinomialTest.(PhFCcountscp.edited, PhFCcountscp.total, 0.05); tail = :right)
#use BenjaminiHochberg to adjust the individual p-value for each gene to keep the overall error rate (or false positive rate) to less than or equal to the user-specified p-value cutoff or error rate.
PhFCcountscp.bBH = adjust(PhFCcountscp.binomial, BenjaminiHochberg());

#use innerjoin function to 'join' Ph1Ccounts and Ph2Ccounts at the position column.
#makeunique=true, duplicate names will be suffixed with _i (i=1 for the first duplicate).
allPhbcp = innerjoin(Ph1Ccountscp, Ph2Ccountscp, on = :pos, makeunique=true)
#use innerjoin function to 'join' allPhb(Ph1Ccounts, Ph2Ccounts) and Ph3Ccounts at the position column.
#makeunique=true, duplicate names will be suffixed with _i (i=2 for the second duplicate).
allPhbcp = innerjoin(allPhbcp, Ph3Ccountscp, on = :pos, makeunique=true)
#use innerjoin function to 'join' allPhbCcounts(Ph1Ccounts, Ph2Ccounts, Ph3Ccounts) and Ph4 at the position column.
#makeunique=true, duplicate names will be suffixed with _i (i=3 for the third duplicate).
allPhbcp = innerjoin(allPhbcp, PhFCcountscp, on = :pos, makeunique=true)
#sum allPhb.bBH_1, allPhb.bBH_2, allPhb.bBH_3 (with a p-value less than 0.05).
sum((allPhbcp.bBH .< 0.05) .& (allPhbcp.bBH_1 .< 0.05) .& (allPhbcp.bBH_2 .< 0.05) .& (allPhbcp.bBH_3 .< 0.05))

LoadError: UndefVarError: Ph1Ccountscp not defined

In [33]:
#make a scatter plot from the DataFrame containing allPh.fBH_1, allPh.fBH_2, allPh.fBH_3 (with a p-value less than 0.05). 
#The scatter plot will consist of position and proportion edited. 
#The ms (marker size), msw (marker stroke width) and alpha (softens colour) are set, there is no legend
# the x and y axes are defined as: x axis="genome position", y axis="proportion of edited transcripts"
editing_sites_binomial_plot = @df allPhbcp[(allPhbcp.bBH .< 0.05) .& (allPhbcp.bBH_1 .< 0.05) .& (allPhbcp.bBH_2 .< 0.05) .& (allPhbcp.bBH_3 .< 0.05),:] scatter(:pos, :editp, ms=4, msw=0, 
    alpha=2, legend=false, xguide="Chloroplast Genome Position", yguide="Proportion of Edited Transcripts");

LoadError: UndefVarError: allPhbcp not defined

In [34]:
savefig("editing_sites_C-U_binomial_sites.svg")

LoadError: No current plot/subplot

In [35]:
#editing sites are defined as all sites in Ph1, Ph2, Ph3 and PhF: All editing sites after adjusted (Benjamini-Hochberg), p-value less than 0.05.
editing_sites_binomial = allPhbcp[(allPhbcp.bBH .< 0.05) .& (allPhbcp.bBH_1 .< 0.05) .& (allPhbcp.bBH_2 .< 0.05) .& (allPhbcp.bBH_3 .< 0.05),:];

LoadError: UndefVarError: allPhbcp not defined

In [36]:
CSV.write("editing_sites_C-U_cp_binomial.csv", editing_sites_binomial)

LoadError: UndefVarError: editing_sites_binomial not defined